In [24]:
import geopandas as gpd
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

import geemap
import ee

In [25]:
ee.Authenticate()
ee.Initialize(project='twoearthengineproject')

In [29]:
# Load AOI shapefile
physio_koshi = gpd.read_file(r"G:\learn\python_works\Work_projects\Koshi_chetan\chetan_cas\works\data\gis\koshi_basin_nepal\koshi_nepal_basin.shp")
whole_koshi_basin = physio_koshi.dissolve()
ee_physio_koshi = ee.FeatureCollection(geemap.geopandas_to_ee(whole_koshi_basin))

# Load Landsat collections
l4 = ee.ImageCollection("LANDSAT/LT04/C02/T1_L2")
l5 = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")
l7 = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2")
l8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
l9 = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")

# Cloud masking functions
def mask_tm(image):
    qa = image.select('QA_PIXEL')
    mask = qa.bitwiseAnd(1 << 3).eq(0).And(qa.bitwiseAnd(1 << 4).eq(0))
    return image.updateMask(mask).multiply(0.0000275).add(-0.2)

def mask_oli(image):
    qa = image.select('QA_PIXEL')
    mask = qa.bitwiseAnd(1 << 3).eq(0).And(qa.bitwiseAnd(1 << 4).eq(0))
    return image.updateMask(mask).multiply(0.0000275).add(-0.2)

# Composite function
def get_composite(year):
    start = ee.Date.fromYMD(year, 1, 1)
    end = ee.Date.fromYMD(year, 12, 31)

    if year < 2013:
        col = (l4.filterBounds(ee_physio_koshi).filterDate(start, end).map(mask_tm)
               .merge(l5.filterBounds(ee_physio_koshi).filterDate(start, end).map(mask_tm))
               .merge(l7.filterBounds(ee_physio_koshi).filterDate(start, end).map(mask_tm)))
    else:
        col = (l8.filterBounds(ee_physio_koshi).filterDate(start, end).map(mask_oli)
               .merge(l9.filterBounds(ee_physio_koshi).filterDate(start, end).map(mask_oli)))

    # Mosaic fills in all available pixels
    return col.mosaic().clip(ee_physio_koshi)

# Years to process
years = [1990, 2000, 2010, 2020, 2024]

# Create map
Map = geemap.Map()
Map.centerObject(ee_physio_koshi, 7)

for y in years:
    img = get_composite(y)
    
    # Add layer for visualization
    if y < 2013:
        vis_params = {'bands': ['SR_B5', 'SR_B4', 'SR_B3'], 'min': 0, 'max': 0.3}
    else:
        vis_params = {'bands': ['SR_B6', 'SR_B5', 'SR_B4'], 'min': 0, 'max': 0.3}
    Map.addLayer(img, vis_params, f"Landsat_{y}")

    # Export all bands
    task = ee.batch.Export.image.toDrive(
        image=img,
        description=f"Landsat_{y}_FullYear",
        folder="GEE_Landsat",
        fileNamePrefix=f"landsat_{y}_full",
        region=ee_physio_koshi.geometry(),
        scale=30,
        maxPixels=1e13
    )
    task.start()

Map

Map(center=[27.459741378291714, 86.82980172772386], controls=(WidgetControl(options=['position', 'transparent_…

In [23]:
# Load WWF HydroSHEDS Free Flowing Rivers
dataset = ee.FeatureCollection('WWF/HydroSHEDS/v1/FreeFlowingRivers')

# Clip dataset to basin
dataset_clipped = dataset.filterBounds(ee_physio_koshi)

# Paint 'RIV_ORD' values
# Filter only RIV_ORD == 5
dataset_order5 = dataset_clipped.filter(ee.Filter.gte('RIV_ORD', 5))

datasetVis = ee.Image().byte().paint(dataset_order5, 'RIV_ORD', 5)

# Visualization parameters
visParams = {
    'min': 5,
    'max': 10,
    'palette': ['08519c', '3182bd', '6baed6', 'bdd7e7', 'eff3ff']
}

# Create geemap Map
Map = geemap.Map()
Map.centerObject(ee_physio_koshi, 8)

# Add layers
Map.addLayer(datasetVis, visParams, 'Free flowing rivers')
Map.addLayer(dataset_clipped, {}, 'Rivers features', False)

Map




Map(center=[27.459741378291714, 86.82980172772386], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
'''# Export clipped rivers to Google Drive as Shapefile
task = ee.batch.Export.table.toDrive(
    collection=dataset_clipped,
    description='FreeFlowingRivers_Koshi',
    folder='GEE_channels_exports',
    fileFormat='SHP'
)
task.start()
print('Export started. Check Google Drive in the folder "GEE_exports" after completion.')'''

In [11]:
# Define export parameters
task = ee.batch.Export.image.toDrive(
    image=datasetVis.clip(ee_physio_koshi),
    description='koshi_rivers_map',
    folder='EarthEngine_Exports',   # Folder in Google Drive
    fileNamePrefix='koshi_rivers',
    region=ee_physio_koshi.geometry(),
    scale=30,                       # Landsat-scale resolution
    crs='EPSG:4326',
    maxPixels=1e13
)

# Start the export task
task.start()

print("Export started. Check your Google Drive folder 'EarthEngine_Exports' in a few minutes.")


Export started. Check your Google Drive folder 'EarthEngine_Exports' in a few minutes.


In [16]:
import ee
import geemap
import geopandas as gpd

# Initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='twoearthengineproject')

# 1. Load Koshi Basin boundary
physio_koshi = gpd.read_file(
    r"G:\learn\python_works\Work_projects\Koshi_chetan\chetan_cas\works\data\gis\koshi_basin_nepal\koshi_nepal_basin.shp"
)
whole_koshi_basin = physio_koshi.dissolve()
ee_physio_koshi = ee.FeatureCollection(geemap.geopandas_to_ee(whole_koshi_basin))

# 2. Function to mask clouds for Landsat Collection 2 Level 2
def maskLandsatL2(image):
    qaPixel = image.select('QA_PIXEL')
    cloudShadowBitMask = 1 << 3
    cloudsBitMask = 1 << 4
    mask = qaPixel.bitwiseAnd(cloudShadowBitMask).eq(0).And(
           qaPixel.bitwiseAnd(cloudsBitMask).eq(0))
    return image.updateMask(mask)

# 3. Function to compute NDWI
def calc_ndwi(image, green_band, nir_band):
    ndwi = image.normalizedDifference([green_band, nir_band]).rename('NDWI')
    return image.addBands(ndwi)

# 4. Function to process one year
def process_year(year):
    if year < 2013:
        # Landsat 5
        collection = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2") \
            .filterBounds(ee_physio_koshi) \
            .filterDate(f"{year}-10-01", f"{year}-11-30") \
            .filter(ee.Filter.lt("CLOUD_COVER", 5)) \
            .map(maskLandsatL2) \
            .map(lambda img: calc_ndwi(img, 'SR_B2', 'SR_B4'))
    else:
        # Landsat 8 & 9
        collection = (ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
                      .merge(ee.ImageCollection("LANDSAT/LC09/C02/T1_L2"))) \
            .filterBounds(ee_physio_koshi) \
            .filterDate(f"{year}-10-01", f"{year}-11-30") \
            .filter(ee.Filter.lt("CLOUD_COVER", 5)) \
            .map(maskLandsatL2) \
            .map(lambda img: calc_ndwi(img, 'SR_B3', 'SR_B5'))

    # Mosaic for the season
    mosaic = collection.median().clip(ee_physio_koshi)
    
    # Threshold NDWI
    water_mask = mosaic.select('NDWI').gt(0).selfMask()

    # Convert raster to vector polygons
    vectors = water_mask.reduceToVectors(
        geometry=ee_physio_koshi.geometry(),
        scale=30,
        geometryType='polygon',
        eightConnected=False,
        labelProperty='class',
        bestEffort=True,
        maxPixels=1e13
    )
    return vectors

# 5. Years to process
years = [1990, 2000, 2010, 2020, 2024]

# 6. Loop over years, export to Google Drive
for year in years:
    print(f"Processing year {year}...")
    river_polygons = process_year(year)
    
    geemap.ee_export_vector_to_drive(
        river_polygons,
        description=f"koshi_rivers_{year}",
        folder="GEE_exports2",
        fileFormat="SHP"
    )
    print(f"Export task for {year} started. Check Google Drive after completion.")

print("All export tasks submitted to Earth Engine.")


Processing year 1990...
Exporting koshi_rivers_1990... Please check the Task Manager from the JavaScript Code Editor.
Export task for 1990 started. Check Google Drive after completion.
Processing year 2000...
Exporting koshi_rivers_2000... Please check the Task Manager from the JavaScript Code Editor.
Export task for 2000 started. Check Google Drive after completion.
Processing year 2010...
Exporting koshi_rivers_2010... Please check the Task Manager from the JavaScript Code Editor.
Export task for 2010 started. Check Google Drive after completion.
Processing year 2020...
Exporting koshi_rivers_2020... Please check the Task Manager from the JavaScript Code Editor.
Export task for 2020 started. Check Google Drive after completion.
Processing year 2024...
Exporting koshi_rivers_2024... Please check the Task Manager from the JavaScript Code Editor.
Export task for 2024 started. Check Google Drive after completion.
All export tasks submitted to Earth Engine.
